In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv


load_dotenv()

In [ ]:
from igraph import compare_communities


def load_sparse_adjacency_matrix(filepath):
    data = np.genfromtxt(os.environ.get("HW6_3B_DATSET") + "sparse_adj.txt", delimiter=",", dtype=int)
    nodes = np.unique(data[:, :2])
    node_to_idx = {node: idx for idx, node in enumerate(nodes)}
    size = len(nodes)

    A = np.zeros((size, size))
    for u, v, w in data:
        i, j = node_to_idx[u], node_to_idx[v]
        A[i, j] = w
        A[j, i] = w
    return A, node_to_idx


def compute_modularity_matrix(mat):
    k = np.sum(mat, axis=1)
    m = np.sum(k) / 2
    B = mat - np.outer(k, k) / (2 * m)
    return B, k, m


def spectral_partitioning(B):
    eigvals, eigvecs = np.linalg.eigh(B)
    v1 = eigvecs[:, -1]
    s = np.where(v1 >= 0, 1, -1)
    return s


def compute_modularity(A, s, k, m):
    s = s.reshape(-1, 1)
    Q = (s.T @ (A - np.outerk(k, k) / (2 * m)) @s) / (4 * m)
    return Q[0, 0]


def modularity_split(filepath):
    A, node_to_idx = load_sparse_adjacency_matrix(filepath=filepath)
    B, k, m = compute_modularity_matrix(A)
    s = spectral_partitioning(B)
    Q = compute_modularity(A, s, k, m)

    idx_to_node = {i: n for n, i in node_to_idx.items()}
    community_1 = [idx_to_node[i] for i in range(len(s)) if s[i] == 1]
    community_2 = [idx_to_node[i] for i in range(len(s)) if s[i] == -1]

    return s, Q, community_1, community_2, A, node_to_idx

In [ ]:
filepath = os.environ.get("HW6_3B_DATSET") + "sparse_adj.txt"
s, Q, com1, com2, A, node_to_idx = modularity_split(filepath)
print(f"Modularity Score: {Q:.4f}")
print("Community 1: ", com1)
print("Community 2: ", com2)